# Load Data

In [1]:
import numpy as np
data = np.load(open('./TaxiBJ21.npy', 'rb'))  # the data file is in the same folder

In [2]:
data.shape

(4272, 2, 32, 32)

Here we simply use the historical data as the input features. External factors, e.g., weather, calendar info, are not used yet. 

In [3]:
# history_length is timestep of the historical window size
# predict_length is the prediction horizon
def generate_data(data, history_length, predict_length):
    history_data = []
    predict_data = []
    total_length = data.shape[0]
    for end_idx in range(history_length + predict_length, total_length):
        predict_frames = data[end_idx-predict_length:end_idx]
        history_frames = data[end_idx-predict_length-history_length:end_idx-predict_length]
        history_data.append(history_frames)
        predict_data.append(predict_frames)
    history_data = np.stack(history_data)
    predict_data = np.stack(predict_data)
    return history_data, predict_data

In [4]:
history_length = 4 * 2  # use the last 4 hours data
predict_length = 1  # predict the next 30 minutes
history_data, predict_data = generate_data(data, history_length, predict_length)
print(history_data.shape)
print(predict_data.shape)

(4263, 8, 2, 32, 32)
(4263, 1, 2, 32, 32)


Split the data with train:valid:test = 8:1:1. Simply, both the validation and testing sets have a data range of 9 days. The other data are used as training set.

In [5]:
history_data_train = history_data[:-18 * 48]
predict_data_train = predict_data[:-18 * 48]

history_data_valid = history_data[-18 * 48:-9 * 48]
predict_data_valid = predict_data[-18 * 48:-9 * 48]

history_data_test = history_data[-9 * 48:]
predict_data_test = predict_data[-9 * 48:]

# A Baseline MLP model

Reshape the data format:

In [6]:
history_data_train = history_data_train.reshape(history_data_train.shape[0], -1)
history_data_train.shape

(3399, 16384)

In [7]:
predict_data_train = predict_data_train.reshape(predict_data_train.shape[0], -1)
predict_data_train.shape

(3399, 2048)

In [8]:
history_data_valid = history_data_valid.reshape(history_data_valid.shape[0], -1)
history_data_valid.shape

(432, 16384)

In [9]:
predict_data_valid = predict_data_valid.reshape(predict_data_valid.shape[0], -1)
predict_data_valid.shape

(432, 2048)

In [10]:
history_data_test = history_data_test.reshape(history_data_test.shape[0], -1)
history_data_test.shape

(432, 16384)

In [11]:
predict_data_test = predict_data_test.reshape(predict_data_test.shape[0], -1)
predict_data_test.shape

(432, 2048)

Build the model:

In [12]:
import tensorflow as tf


model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_dim=history_data_train.shape[-1]),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(predict_data_train.shape[-1], activation='sigmoid')
])

In [13]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               8389120   
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_4 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_5 (Dense)              (None, 2048)              1050624   
Total params: 10,490,368
Trainable params: 10,490,368
Non-trainable params: 0
____________________________________________

In [14]:
model.compile(loss="mse", optimizer="adam")

In [15]:
model.fit(
    history_data_train,
    predict_data_train,
    batch_size=10,
    epochs=100,
    validation_data = (history_data_valid, predict_data_valid),
    # callbacks=tf.keras.callbacks.EarlyStopping(patience=10, monitor='val_loss'),
    verbose=1)

Epoch 1/100
340/340 [==============================] - 1s 4ms/step - loss: 0.0084 - val_loss: 0.0021
Epoch 2/100
340/340 [==============================] - 1s 3ms/step - loss: 0.0012 - val_loss: 0.0017
Epoch 3/100
340/340 [==============================] - 1s 3ms/step - loss: 8.3043e-04 - val_loss: 9.8986e-04
Epoch 4/100
340/340 [==============================] - 1s 3ms/step - loss: 6.1202e-04 - val_loss: 8.2452e-04
Epoch 5/100
340/340 [==============================] - 1s 3ms/step - loss: 6.0375e-04 - val_loss: 8.6686e-04
Epoch 6/100
340/340 [==============================] - 1s 3ms/step - loss: 5.6234e-04 - val_loss: 7.7167e-04
Epoch 7/100
340/340 [==============================] - 1s 3ms/step - loss: 5.2031e-04 - val_loss: 7.2740e-04
Epoch 8/100
340/340 [==============================] - 1s 3ms/step - loss: 4.4807e-04 - val_loss: 6.1712e-04
Epoch 9/100
340/340 [==============================] - 1s 3ms/step - loss: 4.3057e-04 - val_loss: 6.6203e-04
Epoch 10/100
340/340 [=============

In [16]:
predict_results = model.predict(history_data_test)

In [17]:
predict_results.shape

(432, 2048)

In [18]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

Evalute RMSE:

In [19]:
np.sqrt(mean_squared_error(predict_results.flatten(), predict_data_test.flatten()))

0.02654992471867174

Evaluate MAE:

In [20]:
mean_absolute_error(predict_results.flatten(), predict_data_test.flatten())

0.011686256842014228